In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
from typing import Dict, List, Tuple, Union
from peft import PeftModel
import torch
from transformers import LlamaTokenizer, AutoModelForCausalLM, GenerationConfig
from flask import Flask, render_template, request
import sys
sys.path.append("../src")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /nfs/v100-022/jiyunjie/anaconda3/envs/hwg001/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /nfs/a100-006/hanweiguang/local/cuda-11.8/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /nfs/v100-022/jiyunjie/anaconda3/envs/hwg001/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/nfs/v100-022/jiyunjie/anaconda3/envs/hwg001/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /nfs/v100-022/jiyunjie/anaconda3/envs/hwg001 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/nfs/v100-022/jiyunjie/anaconda3/envs/hwg001/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


In [2]:
model_name_or_path='/nfs/a100-80G-17/jiyunjie/finetuned_ckpt/on_belle_tokenizer50k_openinstr_zh/zh_alpaca_gpt3.5_gpt4_sharegpt_epoch=2-step=20652'
ckpt_path='/nfs/a100-006/hanweiguang/saved_model/boxue_1_bs-8_lr-3e-4_wm-1e-2_epoch-10_lora'

In [4]:
# Initialize the model and tokenizer
load_type = torch.float16
tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token_id = 0
tokenizer.bos_token_id = 1
tokenizer.eos_token_id = 2
tokenizer.padding_side = "left"

In [4]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=load_type
)
model = PeftModel.from_pretrained(
    base_model, ckpt_path, torch_dtype=load_type
)
model.eval()
model.to('cuda:0')
print('Model loaded!')

Model loaded!


In [6]:
def get_probability_per_token(input_ids: torch.Tensor) -> Tuple[List[float], float]:
    """
    input_ids: [1, sen_len]
    """
    with torch.no_grad():
        outputs = model(input_ids=input_ids, labels=input_ids)
        logits = outputs.logits
        probs = logits.softmax(dim=-1)
    # Select the probabilities of the generated tokens
    # [1, sen_len, 1]
    generated_token_probs = torch.gather(
        probs[:, :-1, :], 2, input_ids[:, 1:, None])  # skip the first token_id
    return generated_token_probs[0, ..., 0].tolist(), outputs.loss.item()

# Generate测试

In [ ]:
generation_config = GenerationConfig(
    temperature=0.3,
    top_p=0.85,
    top_k=30,
    num_beams=1,
    bos_token_id=1,
    eos_token_id=2,
    pad_token_id=0,
    max_new_tokens=200,  # max_length=max_new_tokens+input_sequence
    min_new_tokens=1,  # min_length=min_new_tokens+input_sequence
    repetition_penalty=1.2,
    do_sample=True,
)

In [ ]:
def decode_and_get_probability(input_text: str, generation_config: GenerationConfig):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        getattr(model, 'module', model).device)
    with torch.no_grad():
        output = model.generate(input_ids=input_ids,
                                generation_config=generation_config)
    token_ids = output[0].tolist()
    tokens = [tokenizer.decode([token_id], skip_special_tokens=False)
              for token_id in token_ids]
    token_probs_list, loss = get_probability_per_token(output)

    # The probability of <s> is set to 0
    tokens_with_probs = [{'token': tokens[0], 'prob': 0.0}]
    for token, prob in zip(tokens[1:], token_probs_list):
        tokens_with_probs.append({
            'token': token,
            'prob': prob
        })

    return tokens_with_probs, loss

In [ ]:
decode_and_get_probability('你是谁', generation_config)

# 计算训练集的loss

In [9]:
import json
with open('/nfs/a100-006/hanweiguang/Projects/BELLE/data/boxue/corpus_1_train.jsonl') as f:
    corpus_1 = f.readlines()
    corpus_1 = [json.loads(s)['text'] for s in corpus_1]

In [32]:
model = PeftModel.from_pretrained(
    base_model, ckpt_path, torch_dtype=load_type
)
model.to('cuda:0')

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(79458, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(
                in_features=4096, out_features=4096, bias=False
      

In [33]:
from tqdm import tqdm
model.eval()
losses = [
    get_probability_per_token(
        tokenizer.encode(
            sample, 
            return_tensors='pt'
        ).to(
            getattr(model, 'module', model).device
        )
    )[1]
    for sample in tqdm(corpus_1)
]

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:47<00:00,  8.36it/s]


In [76]:
import numpy as np
np.mean(losses)

4.13150146484375

# 验证zero 3 + lora保存的模型是否带上了lora

In [5]:
model_name_or_path='/nfs/a100-80G-17/jiyunjie/finetuned_ckpt/on_belle_tokenizer50k_openinstr_zh/zh_alpaca_gpt3.5_gpt4_sharegpt_epoch=2-step=20652'
ckpt_path='/nfs/a100-006/hanweiguang/saved_model/boxue_1_bs-32_lr-3e-4_wm-1e-2_epoch-100_lora'
base_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=load_type
)
model = PeftModel.from_pretrained(
    base_model, ckpt_path, torch_dtype=load_type
)
model.eval()
model.to('cuda:0')
print('Model loaded!')

Model loaded!


In [7]:
import json
with open('/nfs/a100-006/hanweiguang/Projects/BELLE/data/boxue/corpus.jsonl') as f:
    corpus = f.readlines()
    corpus = [json.loads(s)['text'] for s in corpus]

In [9]:
from tqdm import tqdm
model.eval()
losses = [
    get_probability_per_token(
        tokenizer.encode(
            sample, 
            return_tensors='pt'
        ).to(
            getattr(model, 'module', model).device
        )
    )[1]
    for sample in tqdm(corpus)
]

100%|██████████████████████████████████████████████████████| 297/297 [00:47<00:00,  6.27it/s]


In [11]:
import numpy as np
np.mean(losses)

4.12377683080808

In [12]:
model = base_model

In [13]:
from tqdm import tqdm
model.eval()
losses = [
    get_probability_per_token(
        tokenizer.encode(
            sample, 
            return_tensors='pt'
        ).to(
            getattr(model, 'module', model).device
        )
    )[1]
    for sample in tqdm(corpus)
]

100%|██████████████████████████████████████████████████████| 297/297 [00:46<00:00,  6.34it/s]


In [14]:
import numpy as np
np.mean(losses)

4.12377683080808

In [ ]:
lora_parameters = torch.load('/nfs/a100-006/hanweiguang/saved_model/boxue_debug_debug/checkpoint-1/adapter_model.bin', map_location='cpu')
lora_parameters.keys()

In [31]:
lora_parameters

{'base_model.model.model.layers.0.self_attn.q_proj.lora_A.weight': tensor([[ 0.0080, -0.0104, -0.0047,  ..., -0.0059,  0.0042,  0.0138],
         [-0.0001,  0.0142,  0.0116,  ...,  0.0026, -0.0109, -0.0138],
         [-0.0013,  0.0007, -0.0032,  ..., -0.0151, -0.0011,  0.0013],
         ...,
         [-0.0083, -0.0054,  0.0052,  ..., -0.0090,  0.0097,  0.0013],
         [-0.0129,  0.0059,  0.0037,  ..., -0.0035,  0.0016, -0.0118],
         [ 0.0020,  0.0120,  0.0019,  ..., -0.0076,  0.0022,  0.0102]],
        dtype=torch.float16),
 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.weight': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float16),
 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.weight': tensor([[-0.0147, -0.0044,  0.0051,  ..., -0.01

In [27]:
list(torch.load('/nfs/a100-006/hanweiguang/saved_model/boxue_1_bs-8_lr-3e-4_wm-1e-2_epoch-10_lora/adapter_model.bin').values())[0].dtype

torch.float32

In [32]:
list(lora_parameters.values())[0].dtype

torch.float16